<a href="https://colab.research.google.com/github/Vineet3693/mext-exam-_-projects/blob/main/Handwritten_Digit_Generation_Web_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Set Up Development Environment

In [ ]:
# For training (Google Colab)
!pip install torch torchvision streamlit

# For web app
!pip install streamlit torch torchvision pillow numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 


Build and Train the Generative Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform
)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# Define Generator (GAN approach)
class Generator(nn.Module):
    def __init__(self, latent_dim=100, num_classes=10):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.num_classes = num_classes

        # Embedding for class labels
        self.label_embedding = nn.Embedding(num_classes, num_classes)

        self.model = nn.Sequential(
            nn.Linear(latent_dim + num_classes, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Embed labels
        label_embedding = self.label_embedding(labels)
        # Concatenate noise and label embedding
        input_tensor = torch.cat([noise, label_embedding], dim=1)
        img = self.model(input_tensor)
        img = img.view(img.size(0), 1, 28, 28)
        return img

# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self, num_classes=10):
        super(Discriminator, self).__init__()
        self.num_classes = num_classes

        # Embedding for class labels
        self.label_embedding = nn.Embedding(num_classes, num_classes)

        self.model = nn.Sequential(
            nn.Linear(28*28 + num_classes, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        # Flatten image
        img_flat = img.view(img.size(0), -1)
        # Embed labels
        label_embedding = self.label_embedding(labels)
        # Concatenate image and label embedding
        input_tensor = torch.cat([img_flat, label_embedding], dim=1)
        validity = self.model(input_tensor)
        return validity

# Initialize models
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss function and optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training loop
num_epochs = 50
latent_dim = 100

for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        batch_size = imgs.shape[0]

        # Ground truth labels
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Move to device
        real_imgs = imgs.to(device)
        labels = labels.to(device)

        # Train Discriminator
        optimizer_D.zero_grad()

        # Real images
        real_validity = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(real_validity, real_labels)

        # Fake images
        noise = torch.randn(batch_size, latent_dim).to(device)
        fake_labels_gen = torch.randint(0, 10, (batch_size,)).to(device)
        fake_imgs = generator(noise, fake_labels_gen)
        fake_validity = discriminator(fake_imgs.detach(), fake_labels_gen)
        d_fake_loss = adversarial_loss(fake_validity, fake_labels)

        d_loss = (d_real_loss + d_fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()

        fake_validity = discriminator(fake_imgs, fake_labels_gen)
        g_loss = adversarial_loss(fake_validity, real_labels)

        g_loss.backward()
        optimizer_G.step()

        if i % 100 == 0:
            print(f"Epoch {epoch}/{num_epochs}, Batch {i}, D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

# Save the trained generator
torch.save(generator.state_dict(), 'generator.pth')

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 483kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.48MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.99MB/s]


Epoch 0/50, Batch 0, D Loss: 0.6886, G Loss: 0.7271
Epoch 0/50, Batch 100, D Loss: 0.7662, G Loss: 0.7019
Epoch 0/50, Batch 200, D Loss: 0.5505, G Loss: 1.1444
Epoch 0/50, Batch 300, D Loss: 0.4785, G Loss: 1.2735
Epoch 0/50, Batch 400, D Loss: 0.4033, G Loss: 2.2358
Epoch 1/50, Batch 0, D Loss: 0.3522, G Loss: 1.4101
Epoch 1/50, Batch 100, D Loss: 0.6601, G Loss: 3.6967
Epoch 1/50, Batch 200, D Loss: 0.1482, G Loss: 3.1093
Epoch 1/50, Batch 300, D Loss: 0.3639, G Loss: 2.8356
Epoch 1/50, Batch 400, D Loss: 0.5522, G Loss: 5.4513
Epoch 2/50, Batch 0, D Loss: 0.2286, G Loss: 5.6320
Epoch 2/50, Batch 100, D Loss: 0.1828, G Loss: 2.7891
Epoch 2/50, Batch 200, D Loss: 0.8369, G Loss: 2.8640
Epoch 2/50, Batch 300, D Loss: 0.2811, G Loss: 2.0585
Epoch 2/50, Batch 400, D Loss: 0.1643, G Loss: 3.5201
Epoch 3/50, Batch 0, D Loss: 0.1409, G Loss: 3.0687
Epoch 3/50, Batch 100, D Loss: 0.2791, G Loss: 2.4494
Epoch 3/50, Batch 200, D Loss: 0.1450, G Loss: 3.4182
Epoch 3/50, Batch 300, D Loss: 0.187


 Streamlit Web Application

In [ ]:
import streamlit as st
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import io

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the same Generator class as in training
class Generator(nn.Module):
    def __init__(self, latent_dim=100, num_classes=10):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.num_classes = num_classes

        self.label_embedding = nn.Embedding(num_classes, num_classes)

        self.model = nn.Sequential(
            nn.Linear(latent_dim + num_classes, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        label_embedding = self.label_embedding(labels)
        input_tensor = torch.cat([noise, label_embedding], dim=1)
        img = self.model(input_tensor)
        img = img.view(img.size(0), 1, 28, 28)
        return img

# Load the trained model
@st.cache_resource
def load_model():
    generator = Generator().to(device)
    generator.load_state_dict(torch.load('generator.pth', map_location=device))
    generator.eval()
    return generator

def generate_images(generator, digit, num_images=5):
    with torch.no_grad():
        # Create random noise
        noise = torch.randn(num_images, 100).to(device)
        # Create labels for the selected digit
        labels = torch.full((num_images,), digit).to(device)
        # Generate images
        fake_images = generator(noise, labels)
        # Convert to numpy and denormalize
        fake_images = fake_images.cpu().numpy()
        fake_images = (fake_images + 1) / 2  # Denormalize from [-1, 1] to [0, 1]
        return fake_images

def array_to_pil(img_array):
    # Convert numpy array to PIL Image
    img_array = (img_array * 255).astype(np.uint8)
    img = Image.fromarray(img_array.squeeze(), mode='L')
    return img

# Streamlit UI
st.title("Handwritten Digit Generator")
st.write("Generate handwritten digits using a trained GAN model")

# Load model
try:
    generator = load_model()
    st.success("Model loaded successfully!")
except:
    st.error("Could not load the model. Please ensure 'generator.pth' is available.")
    st.stop()

# Digit selection
selected_digit = st.selectbox("Select a digit to generate:", list(range(10)))

# Generate button
if st.button("Generate 5 Images"):
    with st.spinner("Generating images..."):
        # Generate images
        generated_images = generate_images(generator, selected_digit, 5)

        # Display images in a row
        cols = st.columns(5)
        for i, col in enumerate(cols):
            with col:
                pil_img = array_to_pil(generated_images[i])
                st.image(pil_img, caption=f"Generated {selected_digit}", width=100)

# Instructions
st.markdown("---")
st.markdown("### Instructions:")
st.markdown("1. Select a digit (0-9) from the dropdown")
st.markdown("2. Click 'Generate 5 Images' to create new handwritten digits")
st.markdown("3. The app will display 5 different variations of the selected digit")

2025-06-22 17:12:30.645 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 17:12:30.753 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-22 17:12:30.754 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 17:12:30.756 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 17:12:30.757 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 17:12:30.759 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 17:12:30.761 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 17:12:30.763 Thread 'MainThread': mi

DeltaGenerator()